In [24]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-02-21 22:35:29--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2026-02-21 22:35:29 (21.8 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



# Building a primitive llm

In [25]:
#get file encoding, ascii is part of utf-8
import chardet

with open ('input.txt', "rb") as f:
  enc=chardet.detect(f.read())
  enc_res=enc["encoding"]
  enc_cof=enc["confidence"]


In [26]:
with open("input.txt", "r", encoding=enc_res) as f:
  text=f.read()
print(f"The lenght of the dataset in characters: {len(text)}")
print(f"first lines {text[:10000]}")


The lenght of the dataset in characters: 1115394
first lines First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I

In [12]:
print(f"The lenght of the dataset in characters: {len(text)}")
print(f"first lines {text[:10000]}")

The lenght of the dataset in characters: 1115394
first lines First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I

In [29]:
vocab=sorted(list(set(text)))
print(vocab)
print(f"Length of vocabulary {len(vocab):,}")
print(f"Length of text was {len(text):,} ")
print(f"Length of text in words {len(text.split()):,}")

print (" ".join(vocab))

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Length of vocabulary 65
Length of text was 1,115,394 
Length of text in words 202,651

   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z


In [28]:
text_separated=text.split()
print(text[:1000])
print(text_separated[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [32]:
print(list(enumerate(text_separated)))

[(0, 'First'), (1, 'Citizen:'), (2, 'Before'), (3, 'we'), (4, 'proceed'), (5, 'any'), (6, 'further,'), (7, 'hear'), (8, 'me'), (9, 'speak.'), (10, 'All:'), (11, 'Speak,'), (12, 'speak.'), (13, 'First'), (14, 'Citizen:'), (15, 'You'), (16, 'are'), (17, 'all'), (18, 'resolved'), (19, 'rather'), (20, 'to'), (21, 'die'), (22, 'than'), (23, 'to'), (24, 'famish?'), (25, 'All:'), (26, 'Resolved.'), (27, 'resolved.'), (28, 'First'), (29, 'Citizen:'), (30, 'First,'), (31, 'you'), (32, 'know'), (33, 'Caius'), (34, 'Marcius'), (35, 'is'), (36, 'chief'), (37, 'enemy'), (38, 'to'), (39, 'the'), (40, 'people.'), (41, 'All:'), (42, 'We'), (43, "know't,"), (44, 'we'), (45, "know't."), (46, 'First'), (47, 'Citizen:'), (48, 'Let'), (49, 'us'), (50, 'kill'), (51, 'him,'), (52, 'and'), (53, "we'll"), (54, 'have'), (55, 'corn'), (56, 'at'), (57, 'our'), (58, 'own'), (59, 'price.'), (60, "Is't"), (61, 'a'), (62, 'verdict?'), (63, 'All:'), (64, 'No'), (65, 'more'), (66, 'talking'), (67, "on't;"), (68, 'let')

In [33]:
a={'b': [1,2,3],
   'c': [2,3,4]}

In [39]:
print(a)
print(list(a.values()))


{'b': [1, 2, 3], 'c': [2, 3, 4]}
[[1, 2, 3], [2, 3, 4]]


In [50]:
voc_int={ch: i for i, ch in enumerate(vocab)}
print((voc_int))

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [ ]:
print("First 10 key-value pairs in voc_int:")
print(list(voc_int.items())[:10])